In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup

import pandas as pd
import re
import os
import glob
import time
from datetime import datetime as dt

In [236]:
def prepare_webpage(driver,time_sleep=2):
    # check checkbox to show restricted offers
    while True:
        try:
            driver.find_element(By.XPATH, "//div[@id='cookiescript_accept']").click()
            break
        except NoSuchElementException:
            break
        except ElementClickInterceptedException:
            pass
    time.sleep(time_sleep)
    
    # load all offers by clicking on "more" button until it disappears
    try:
        driver.find_element(By.XPATH, "//span[@id='showAll']").click()
        time.sleep(time_sleep)
    except:
        pass

In [256]:
def scrape_offers(html, maturity=''):
    soup = BeautifulSoup(html, 'lxml')
    bank_names = [name['alt'] for name in soup.find_all('img', {'loading': 'lazy'})]
    interest_rates = [rate.text.strip() for rate in soup.find_all('span', {'class': 'rente'})]
    countries = [country.text.split(maxsplit=1)[1] for country in soup.find_all('div', {'class': 'block-kenmerk block-garantie'})]
    
    if maturity=='':
        maturity = [term.text.split(maxsplit=1)[1] for term in soup.find_all('div', {'class': 'block-kenmerk second'})]
    
    df = pd.DataFrame({'bank_name': bank_names,
                       'country': countries,
                       'maturity_months': maturity,
                      'interest_rate': interest_rates,
                      'timestamp': dt.now().time().replace(microsecond=0)})
    return df

In [5]:
def get_platform(x):
    try:
        if re.compile(r"Savedo").search(x):
            platform = re.compile(r"Savedo").search(x).group(0)
        else:
            platform = re.compile(r"Raisin").search(x).group(0)
    except:
        platform = 'none'
        
    return platform

In [333]:
def translate_maturity(x):
    try:
        if x.split()[-1].strip() == 'jaar':
            maturity_months = int(x.split()[0].strip()) * 12
        else:
            maturity_months = int(x.split()[0].strip())
    except:
        maturity_months = x
            
    return maturity_months

In [334]:
driver = webdriver.Chrome()
driver.maximize_window()

In [335]:
time_sleep = 2

### Savings accounts

In [336]:
url = "https://www.sparen.nl/vergelijken"

In [337]:
driver.get(url)
time.sleep(time_sleep)

In [338]:
prepare_webpage(driver)

In [339]:
savings = scrape_offers(html=driver.page_source, maturity=0)

### Fixed-term deposits

In [340]:
url = "https://www.sparen.nl/vergelijken/deposito"

In [341]:
driver.get(url)
time.sleep(time_sleep)

In [342]:
list_fixed_df = []
terms_count = driver.find_elements(By.XPATH, "//label[contains(@for,'radioLooptijd')]")

In [343]:
for n in range(len(terms_count)):
    term_buttons = driver.find_elements(By.XPATH, "//label[contains(@for,'radioLooptijd')]")
    term_buttons[n].click()
    time.sleep(time_sleep)
    
    prepare_webpage(driver)
    
    fixed_df = scrape_offers(html=driver.page_source)
    list_fixed_df.append(fixed_df)

In [344]:
fixed_normal = pd.concat(list_fixed_df)

### Clean data

In [345]:
df = pd.concat([savings,fixed_normal])

In [346]:
df['interest_rate'] = pd.to_numeric(df['interest_rate'].str.split(' %', expand = True)[0].str.replace(",", "."))

In [347]:
df['country']=df['country'].str.strip()

In [348]:
country_names_eng = {'Duitsland': 'Germany',
                    'Nederland': 'Netherlands', 
                     'België': 'Belgium', 
                     'Letland': 'Latvia', 
                     'Italië': 'Italy', 
                     'Estland': 'Estonia',
                     'Slowakije': 'Slovakia', 
                     'Polen': 'Poland', 
                     'Kroatië': 'Croatia', 
                     'Tsjechië': 'Czech Republic',
                     'Ierland': 'Ireland',
                     'Spanje': 'Spain',
                     'Frankrijk': 'France',
                     'Zweden': 'Sweden',
                     'Litouwen': 'Lithuania',
                    'Oostenrijk': 'Austria'}

missing_country_banks = {'Brand New Day': 'Netherlands',
                        'ING': 'Netherlands',
                        'Evi van Lanschot': 'Netherlands',
                        'DELA': 'Netherlands',
                        'GarantiBank': 'Netherlands', 
                        'ABN AMRO': 'Netherlands', 
                        'Rabobank': 'Netherlands',
                        'Banca Progetto (via Raisin)': 'Italy', 
                        'Coop Pank (via Raisin)': 'Estonia',
                        'Euram Bank (via Raisin)': 'Austria', 
                         'CiviBank (via Raisin)': 'Italy',
                         'North Channel Bank (via Raisin)': 'Germany', 
                         'J&T Banka (via Savedo)': 'Czech Republic'}

In [349]:
df['country'] = df['country'].replace(country_names_eng)

In [350]:
df['platform'] = df['bank_name'].apply(get_platform)

In [351]:
df['date'] = dt.now().date()

In [361]:
df['maturity_months']=df['maturity_months'].apply(translate_maturity)

In [362]:
df.drop_duplicates(inplace = True)

In [363]:
df = df[['date', 'bank_name', 'country', 'maturity_months', 'interest_rate',  'platform', 'timestamp']]

In [364]:
df.sort_values(['bank_name', 'maturity_months'], inplace=True)

### Store as csv file

In [365]:
path = '../Interest Rates/NL_sparen.nl/'
filename=str(dt.now().strftime("%Y%m%d_%H-%M"))  + '_scrape.csv'
df.to_csv(os.path.join(path,filename),encoding='utf-8-sig',index=False)

In [366]:
driver.close()

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF627BB4D02+56194]
	(No symbol) [0x00007FF627B204B2]
	(No symbol) [0x00007FF6279C74C3]
	(No symbol) [0x00007FF627A03F9A]
	(No symbol) [0x00007FF627A35D59]
	(No symbol) [0x00007FF627A2EB12]
	(No symbol) [0x00007FF627A2E44F]
	(No symbol) [0x00007FF627996315]
	GetHandleVerifier [0x00007FF627F2D5BB+3695675]
	GetHandleVerifier [0x00007FF627F86197+4059159]
	GetHandleVerifier [0x00007FF627F7DF63+4025827]
	GetHandleVerifier [0x00007FF627C4F029+687785]
	(No symbol) [0x00007FF627B2B508]
	(No symbol) [0x00007FF627995103]
	GetHandleVerifier [0x00007FF627FCD4A8+4350760]
	BaseThreadInitThunk [0x00007FF8DF45257D+29]
	RtlUserThreadStart [0x00007FF8DF5EAA58+40]


In [367]:
df

,date,bank_name,country,maturity_months,interest_rate,platform,timestamp
34,2024-02-01,ABN AMRO,Netherlands,0,1.50,none,17:02:40
34,2024-02-01,ABN AMRO,Netherlands,12,2.35,none,17:03:05
34,2024-02-01,ABN AMRO,Netherlands,24,2.35,none,17:03:11
32,2024-02-01,ABN AMRO,Netherlands,36,2.35,none,17:03:18
27,2024-02-01,ABN AMRO,Netherlands,60,2.35,none,17:03:31
...,...,...,...,...,...,...,...
26,2024-02-01,Younited (via Raisin),France,24,2.75,Raisin,17:03:11
27,2024-02-01,Younited (via Raisin),France,36,2.55,Raisin,17:03:18
27,2024-02-01,Younited (via Raisin),France,48,2.35,Raisin,17:03:25
29,2024-02-01,Younited (via Raisin),France,60,2.15,Raisin,17:03:31
